# Libraries

In [5]:
from datetime import datetime
from datetime import timedelta
import shutil
import os

In [6]:
import tensorflow as tf

In [7]:
!python --version

Python 3.9.2


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Data Transformer

In [9]:
import numpy as np


class Transformer:

    def __init__(self, decision_size, decision_overlap, segments_size=90, segments_overlap=45, sampling=2):
        self.segments_size = segments_size
        self.segments_overlap = segments_overlap
        self.sampling = sampling
        self.decision_size = decision_size
        self.decision_overlap = decision_overlap

    def transfer(self, dataset, features, method):
        print("segmenting data with " + str(len(dataset)) + " points")
        segments, labels = self.__segment_signal(dataset, features)
        print("making " + str(len(segments)) + " segments")
        if method == "table":
            segments_dataset = self.__transfer_table(segments, features)
        elif method == "1d":
            segments_dataset = self.__transfer_1d(segments, features)
        elif method == "2d":
            segments_dataset = self.__transfer_2d(segments, features)
        elif method == "3d_1ch":
            segments_dataset = self.__transfer_2d_1ch(segments, features)
        elif method == "3d":
            segments_dataset = self.__transfer_3d(segments, features)
        elif method == "4d":
            segments_dataset = self.__transfer_4d(segments, features)
        elif method == "rnn_2d":
            segments_dataset, labels = self.__transfer_rnn_2d(segments, labels, features)
        elif method == "rnn_3d_1ch":
            segments_dataset, labels = self.__transfer_rnn_3d_1ch(segments, labels, features)
        return segments_dataset, labels

    @staticmethod
    def data_shape(method, n_features, segments_size, segments_overlap=None, decision_size=None):
        if method == "table":
            return (None, n_features * segments_size)
        elif method == "1d":
            return (None, 1, n_features * segments_size, 1)
        elif method == "2d":
            return (None, n_features, segments_size)
        elif method == "3d_1ch":
            return (None, n_features, segments_size, 1)
        elif method == "3d":
            return (None, 1, segments_size, n_features)
        elif method == "4d":
            return (n_features, None, 1, segments_size, 1)
        elif method == "rnn_2d":
            s_b = Transformer.get_segments_a_decision_window(segments_size,
                                                             int(segments_size * segments_overlap),
                                                             decision_size)
            return (None, s_b, segments_size * n_features)
        elif method == "rnn_3d_1ch":
            s_b = Transformer.get_segments_a_decision_window(segments_size,
                                                             int(segments_size * segments_overlap),
                                                             decision_size)
            return (None, s_b, 1, segments_size * n_features)
        return ()

    @staticmethod
    def get_segments_a_decision_window(segment_size, segment_overlap_size, decision_size):
        return int((decision_size - segment_size) / (segment_size - segment_overlap_size) + 1)

    def __transfer_rnn_2d(self, segments, labels, features):
        y_output = []
        x_output = []
        c = len(np.unique(labels))
        s = Transformer.get_segments_a_decision_window(self.segments_size,
                                                       self.segments_overlap,
                                                       self.decision_size)
        r = int(np.floor(s * self.decision_overlap))
        for _id in np.unique(labels):
            subset = segments[np.where(labels == _id)]
            n = subset.shape[0]
            o = int(np.floor((n - r) / (s - r)))
            for i in range(o):
                row = []
                for j in range(s):
                    A = subset[i * (s - r) + j]
                    A = A.reshape(A.shape[0] * A.shape[1])
                    row.append(A)
                y_output.append(_id)
                x_output.append(row)
        x_output = np.array(x_output)
        y_output = np.array(y_output)
        return x_output, y_output

    def __transfer_rnn_3d_1ch(self, segments, labels, features):
        # (samples, time, channels=1, rows)
        y_output = []
        x_output = []
        c = len(np.unique(labels))
        s = Transformer.get_segments_a_decision_window(self.segments_size,
                                                       self.segments_overlap,
                                                       self.decision_size)
        r = int(np.floor(s * self.decision_overlap))
        for _id in np.unique(labels):
            subset = segments[np.where(labels == _id)]
            n = subset.shape[0]
            o = int(np.floor((n - r) / (s - r)))
            for i in range(o):
                row = []
                for j in range(s):
                    A = subset[i * (s - r) + j]
                    A = A.reshape(A.shape[0] * A.shape[1])
                    row.append([A])
                y_output.append(_id)
                x_output.append(row)
        x_output = np.array(x_output)
        y_output = np.array(y_output)
        return x_output, y_output

    def __transfer_table(self, segments, features):
        new_dataset = []
        for segment in segments:
            row = []
            for feature_i in range(len(features)):
                for i in range(len(segment[feature_i])):
                    row.append(segment[feature_i][i])
            new_dataset.append(row)

        new_dataset = np.array(new_dataset)
        return new_dataset

    def __transfer_1d(self, segments, features):
        new_dataset = []
        for segment in segments:
            row = []
            for feature_i in range(len(features)):
                for i in range(len(segment[feature_i])):
                    row.append(segment[feature_i][i])
            new_dataset.append([row])

        new_dataset = np.array(new_dataset)
        return np.expand_dims(new_dataset, axis=3)

    def __transfer_2d(self, segments, features):
        new_dataset = []
        for segment in segments:
            row = []
            for feature_i in range(len(features)):
                row.append(segment[feature_i])
            new_dataset.append(row)

        new_dataset = np.array(new_dataset)
        return new_dataset

    def __transfer_3d_1ch(self, segments, features):
        new_dataset = []
        for segment in segments:
            row = []
            for feature_i in range(len(features)):
                row.append(segment[feature_i])
            new_dataset.append(row)

        new_dataset = np.array(new_dataset)
        return np.expand_dims(new_dataset, axis=3)

    def __transfer_3d(self, segments, features):
        new_dataset = []
        for segment in segments:
            row = []
            for i in range(len(segment[0])):
                cell = []
                for feature_i in range(len(features)):
                    cell.append(segment[feature_i][i])
                row.append(cell)
            new_dataset.append([row])

        new_dataset = np.array(new_dataset)
        return new_dataset

    def __transfer_4d(self, segments, features):
        new_dataset = []
        for feature_i in range(len(features)):
            row = []
            for segment in segments:
                cell = []
                for element in segment[feature_i]:
                    cell.append([element])
                row.append([cell])
            new_dataset.append(row)

        new_dataset = np.array(new_dataset)
        return new_dataset

    def __windows(self, data):
        start = 0
        while start < data.count():
            yield int(start), int(start + self.segments_size)
            start += (self.segments_size - self.segments_overlap)

    def __segment_signal(self, dataset, features):
        segments = []
        labels = []
        for class_i in np.unique(dataset["id"]):
            subset = dataset[dataset["id"] == class_i]
            for (start, end) in self.__windows(subset["id"]):
                feature_slices = []
                for feature in features:
                    feature_slices.append(subset[feature][start:end].tolist())
                if len(feature_slices[0]) == self.segments_size:
                    segments.append(feature_slices)
                    labels.append(class_i)
        return np.array(segments), np.array(labels)

# Dataset

In [10]:
import glob
import os
from datetime import timedelta
import pandas as pd
import numpy as np
from sklearn import preprocessing


class Dataset:

    def __init__(self, db_path, sample_rate, features,
                 window_time, window_overlap_percentage,
                 decision_time, decision_overlap_percentage,
                 add_noise, noise_rate,
                 train_blocks: list, valid_blocks: list, test_blocks: list,
                 data_length_time=-1):
        """
        :param db_path:
        :param sample_rate:
        :param features:
        :param window_time: in seconds
        :param window_overlap_percentage: example: 0.75 for 75%
        :param add_noise: True or False
        :param noise_rate:
        :param train_blocks:
        :param valid_blocks:
        :param test_blocks:
        :param data_length_time: the amount of data from each class in seconds. -1 means whole existing data.
        """
        self.db_path = db_path
        self.features = features
        self.sample_rate = sample_rate
        self.window_size = window_time * sample_rate
        self.window_overlap_size = int(self.window_size * window_overlap_percentage)
        self.decision_size = decision_time * sample_rate
        self.decision_overlap_size = int(self.decision_size * decision_overlap_percentage)
        self.decision_overlap_percentage = decision_overlap_percentage
        self.add_noise = add_noise
        self.noise_rate = noise_rate
        self.train_blocks = train_blocks
        self.valid_blocks = valid_blocks
        self.test_blocks = test_blocks
        self.data_length_size = data_length_time * sample_rate if data_length_time != -1 else -1

        # Initialization
        self.train_dataset = pd.DataFrame()
        self.valid_dataset = pd.DataFrame()
        self.test_dataset = pd.DataFrame()
        self.n_train_dataset = pd.DataFrame()
        self.n_valid_dataset = pd.DataFrame()
        self.n_test_dataset = pd.DataFrame()
        self.X_train = np.array([])
        self.y_train = np.array([])
        self.X_valid = np.array([])
        self.y_valid = np.array([])
        self.X_test = np.array([])
        self.y_test = np.array([])

    def load_data(self, n_classes, method, read_cache=True, random_selection=True):
        segments_path = "./" + self.db_path.split("/")[-2] + "/" + \
                        "segments/" + \
                        "method: " + str(method) + os.sep + \
                        "noise: " + str(self.noise_rate) + os.sep + \
                        "n_classes:" + str(n_classes) + os.sep + \
                        "wl: " + str(self.window_size) + os.sep + \
                        "wo: " + str(self.window_overlap_size) + os.sep + \
                        "dl: " + str(self.decision_size) + os.sep + \
                        "do: " + str(self.decision_overlap_size) + os.sep + \
                        "train: " + str(self.train_blocks) + os.sep + \
                        "valid: " + str(self.valid_blocks) + os.sep + \
                        "test: " + str(self.test_blocks) + os.sep
        print(segments_path)
        if read_cache \
                and os.path.exists(segments_path + 'X_train.npy') \
                and os.path.exists(segments_path + 'y_train.npy') \
                and os.path.exists(segments_path + 'X_valid.npy') \
                and os.path.exists(segments_path + 'y_valid.npy') \
                and os.path.exists(segments_path + 'X_test.npy') \
                and os.path.exists(segments_path + 'y_test.npy'):
            print("Dataset is already")
            self.X_train = np.load(segments_path + 'X_train.npy')
            self.y_train = np.load(segments_path + 'y_train.npy')
            self.X_valid = np.load(segments_path + 'X_valid.npy')
            self.y_valid = np.load(segments_path + 'y_valid.npy')
            self.X_test = np.load(segments_path + 'X_test.npy')
            self.y_test = np.load(segments_path + 'y_test.npy')
        else:
            self.__preprocess(n_classes, method, random_selection)
            # Save Dataset
            if not os.path.exists(segments_path):
                os.makedirs(segments_path)
            np.save(segments_path + 'X_train.npy', self.X_train)
            np.save(segments_path + 'y_train.npy', self.y_train)
            np.save(segments_path + 'X_valid.npy', self.X_valid)
            np.save(segments_path + 'y_valid.npy', self.y_valid)
            np.save(segments_path + 'X_test.npy', self.X_test)
            np.save(segments_path + 'y_test.npy', self.y_test)

        def to_dic(data):
            dic = {}
            for i, x in enumerate(data):
                dic[str(i)] = x
            return dic

        if len(self.X_train.shape) == 5:
            self.X_train = to_dic(self.X_train)
            self.X_valid = to_dic(self.X_valid)
            self.X_test = to_dic(self.X_test)

    def __preprocess(self, n_classes, method, random_selection=True):
        if random_selection:
            csv_paths = np.random.choice(glob.glob(self.db_path + "*.csv"), n_classes, replace=False)
        else:
            csv_paths = glob.glob(self.db_path + "*.csv")[:n_classes]

        self.class_names = {}
        for i, csv_path in enumerate(csv_paths):
            label = os.path.basename(csv_path).split('.')[0]
            self.class_names[label] = i
            train, valid, test = self.__read_data(csv_path, self.features, label)
            train['id'] = i
            valid['id'] = i
            test['id'] = i
            self.train_dataset = pd.concat([self.train_dataset, train])
            self.valid_dataset = pd.concat([self.valid_dataset, valid])
            self.test_dataset = pd.concat([self.test_dataset, test])

        self.__standardization()
        self.__segmentation(method=method)

    def __read_data(self, path, features, label):
        data = pd.read_csv(path, low_memory=False)
        data = data[features]
        data = data.fillna(data.mean())
        length = self.data_length_size if self.data_length_size != -1 else data.shape[0]
        print('class: %5s, data size: %s, selected data size: %s' % (
            label, str(timedelta(seconds=int(data.shape[0] / self.sample_rate))),
            str(timedelta(seconds=int(length / self.sample_rate)))))
        return self.__split_to_train_valid_test(data)

    def __split_to_train_valid_test(self, data):
        n_blocks = max(self.train_blocks + self.valid_blocks + self.test_blocks) + 1
        block_length = int(len(data[:self.data_length_size]) / n_blocks)

        train_data = pd.DataFrame()
        for i in range(len(self.train_blocks)):
            start = self.train_blocks[i] * block_length
            end = self.train_blocks[i] * block_length + block_length - 1
            if train_data.empty:
                train_data = data[start:end]
            else:
                train_data = pd.concat([data[start:end], train_data])

        valid_data = pd.DataFrame()
        for i in range(len(self.valid_blocks)):
            start = self.valid_blocks[i] * block_length
            end = self.valid_blocks[i] * block_length + block_length - 1
            if valid_data.empty:
                valid_data = data[start:end]
            else:
                valid_data = pd.concat([data[start:end], valid_data])

        test_data = pd.DataFrame()
        for i in range(len(self.test_blocks)):
            start = self.test_blocks[i] * block_length
            end = self.test_blocks[i] * block_length + block_length - 1
            if test_data.empty:
                test_data = data[start:end]
            else:
                test_data = pd.concat([data[start:end], test_data])

        if self.add_noise:
            test_data = self.__add_noise_to_data(test_data)

        return train_data, valid_data, test_data

    def __add_noise_to_data(self, x):
        x_power = x ** 2
        sig_avg_watts = np.mean(x_power)
        sig_avg_db = 10 * np.log10(sig_avg_watts)
        noise_avg_db = sig_avg_db - self.target_snr_db
        noise_avg_watts = 10 ** (noise_avg_db / 10)
        mean_noise = 0
        noise_volts = np.random.normal(mean_noise, np.sqrt(noise_avg_watts), size=x.shape)
        return x + noise_volts

    def __standardization(self):
        scaler = preprocessing.StandardScaler()
        scaler = scaler.fit(self.train_dataset.iloc[:, :-1])
        n_train_dataset = scaler.transform(self.train_dataset.iloc[:, :-1])
        n_valid_dataset = scaler.transform(self.valid_dataset.iloc[:, :-1])
        n_test_dataset = scaler.transform(self.test_dataset.iloc[:, :-1])

        self.n_train_dataset = pd.DataFrame(n_train_dataset, columns=self.features)
        self.n_valid_dataset = pd.DataFrame(n_valid_dataset, columns=self.features)
        self.n_test_dataset = pd.DataFrame(n_test_dataset, columns=self.features)
        self.n_train_dataset['id'] = self.train_dataset.iloc[:, -1].tolist()
        self.n_valid_dataset['id'] = self.valid_dataset.iloc[:, -1].tolist()
        self.n_test_dataset['id'] = self.test_dataset.iloc[:, -1].tolist()

    def __segmentation(self, method):
        transformer = Transformer(segments_size=self.window_size,
                                  segments_overlap=self.window_overlap_size,
                                  decision_size=self.decision_size,
                                  decision_overlap=self.decision_overlap_percentage)
        self.X_train, self.y_train = transformer.transfer(self.n_train_dataset, self.features, method=method)
        self.X_valid, self.y_valid = transformer.transfer(self.n_valid_dataset, self.features, method=method)
        self.X_test, self.y_test = transformer.transfer(self.n_test_dataset, self.features, method=method)


# Scoring Functions

In [11]:
import tensorflow as tf
import numpy as np


def pesl(y, q):
    y = tf.cast(y, tf.float32)
    c = y.get_shape()[1]

    ST = tf.math.subtract(q, tf.reduce_sum(tf.where(y == 1, q, y), axis=1)[:, None])
    ST = tf.where(ST < 0, tf.constant(0, dtype=tf.float32), ST)
    payoff = tf.reduce_sum(tf.math.ceil(ST), axis=1)
    M = (c - 1) / (c ** 2)
    payoff = tf.where(payoff > 0, tf.constant(M, dtype=tf.float32), payoff)
    return tf.math.reduce_mean(tf.math.reduce_mean(tf.math.square(tf.math.subtract(y, q)), axis=1) + payoff)

In [12]:
def esl(y,q):
    return tf.math.reduce_mean(tf.keras.losses.mean_squared_error(y,q))

In [13]:
import math

def pll(y, q):
    y = tf.cast(y, tf.float32)
    c = y.get_shape()[1]

    ST = tf.math.subtract(q, tf.reduce_sum(tf.where(y == 1, q, y), axis=1)[:, None])
    ST = tf.where(ST < 0, tf.constant(0, dtype=tf.float32), ST)
    payoff = tf.reduce_sum(tf.math.ceil(ST), axis=1)
    M = math.log(1/c)
    payoff = tf.where(payoff > 0, tf.constant(M, dtype=tf.float32), payoff)
    log_loss = tf.keras.losses.categorical_crossentropy(y,q)
    p_log_loss = tf.cast(log_loss, tf.float32) - payoff
    return tf.math.reduce_mean(p_log_loss)

In [14]:
def ll(y,q):
    return tf.math.reduce_mean(tf.keras.losses.categorical_crossentropy(y,q))

# Metrics

In [15]:
import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Define the true positives, false positives and false negatives
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    # Calculate the precision and recall
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    # Calculate the F1 score
    f1_score = 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    return f1_score

# MODELS

In [16]:
class Classifier():
    def __init__(self, loss_function):
        self.loss_function = loss_function

    def get_loss_function(self):
        return self.loss_function

    def get_loss_function_name(self):
        return self.loss_function if type(self.loss_function) == str else self.loss_function.__name__

In [58]:
# Convolutinal Neural Network
class CNN_L(Classifier):
    def __init__(self, classes, n_features,
                 segments_size, segments_overlap,
                 decision_size, decision_overlap,
                 loss_function, loss_metric, lr=0.0001, beta_1=0.5):
        super().__init__(loss_metric)
        self.classes = classes
        self.n_features = n_features
        self.segments_size = segments_size
        self.input_shape = self.get_input_shape()
        self.segments_overlap = segments_overlap
        self.decision_size = decision_size
        self.decision_overlap = decision_overlap
        self.initializer = tf.keras.initializers.GlorotNormal(seed=0)

        # Build and compile the model
        self.model = self.build_model_l()

        # 'categorical_crossentropy'
        optimizer = tf.keras.optimizers.Nadam(lr, beta_1)
        self.model.compile(loss=loss_function,
                           optimizer=optimizer,
                           metrics=['accuracy'])

    def get_input_shape(self):
        data_shape = Transformer.data_shape(method=self.get_data_arrangement(), n_features=self.n_features,
                                            segments_size=self.segments_size)
        return data_shape[-3], data_shape[-2], data_shape[-1]

    @staticmethod
    def get_data_arrangement():
        return "3d"

    def count_params(self):
        return self.model.count_params()

    def build_model_l(self):
        input_ = tf.keras.layers.Input(shape=self.input_shape)

        x = tf.keras.layers.Conv2D(64, kernel_size=(1, 3), strides=1, padding="same", activation='relu', kernel_initializer=self.initializer, bias_initializer=self.initializer)(input_)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.Conv2D(32, kernel_size=(1, 3), strides=1, padding="same", activation='relu', kernel_initializer=self.initializer, bias_initializer=self.initializer)(x)
        x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.Conv2D(16, kernel_size=(1, 3), strides=1, padding="same", activation='relu', kernel_initializer=self.initializer, bias_initializer=self.initializer)(x)
        x = tf.keras.layers.BatchNormalization(momentum=0.9)(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
        x = tf.keras.layers.Dropout(0.2)(x)

        dense = tf.keras.layers.Flatten()(x)
        dense = tf.keras.layers.Dense(1024, kernel_initializer=self.initializer, bias_initializer=self.initializer)(dense)
        dense = tf.keras.layers.LeakyReLU(alpha=0.2)(dense)
        dense = tf.keras.layers.Dropout(0.2)(dense)
        dense = tf.keras.layers.Dense(self.classes, activation='softmax')(dense)

        model = tf.keras.models.Model(inputs=input_, outputs=[dense])
        return model

    def train(self, epochs, X_train, y_train, X_valid, y_valid, X_test, callback, batch_size=128):
        # Change the labels from categorical to one-hot encoding
        y_train_onehot = np.asarray(pd.get_dummies(y_train), dtype=np.int8)
        y_valid_onehot = np.asarray(pd.get_dummies(y_valid), dtype=np.int8)

        history = self.model.fit(X_train, y_train_onehot,
                                 validation_data=(X_valid, y_valid_onehot),
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 verbose=1,
                                 shuffle=True,
                                 callbacks=callback)

        return history, self.model.predict(X_test)

# Callbacks

In [98]:
class ModelAnalyser:

    def __init__(self, segment_size, segment_overlap, decision_size, decision_overlap, X, y):
        """
        :param segment_size: for calculating number of segments in a decision window.
        :param segment_overlap: for calculating number of segments in a decision window.
        :param decision_size:
        :param decision_overlap:
        :param X:
        :param y:
        """
        self.segments_a_decision_window = get_segments_a_decision_window(segment_size, segment_overlap, decision_size)
        self.decision_overlap = decision_overlap
        self.X = X
        self.y_real = y

    def measurement(self, model, monitor):
        """
        :param y_real: expected labels
        :param model: the core
        :param monitor: '#METHOD_#MEASURE', #METHOD={'mv','ms'}, #MEASURE={'loss','accuracy','precision','recall','f1'}
        """
        y_prediction = model.predict(self.X)
        
        monitor_method = monitor.split('_')[0]
        monitor_measure = monitor.split('_')[1]

        if monitor_method == 'ms':
            y_pred_labels, y_dw_real = AveragingProbabilities(y_truth=self.y_real,
                                                              y_prediction=y_prediction,
                                                              s=self.segments_a_decision_window,
                                                              r=self.decision_overlap)

            y_pred_one_hot = np.zeros_like(y_pred_labels)
            y_pred_one_hot[np.arange(len(y_pred_one_hot)), y_pred_labels.argmax(1)] = 1
            if monitor_measure == 'mse':
                loss_fn = tf.keras.losses.MeanSquaredError()
                return loss_fn(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'pesl':
                return pesl(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'pll':
                return pll(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'accuracy':
                return accuracy_score(y_dw_real, y_pred_one_hot)
            elif monitor_measure == 'precision':
                return precision_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'recall':
                return recall_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'f1':
                return f1_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'confusionmatrix':
                y_dw_real_category = [ np.argmax(t) for t in y_dw_real ]
                y_pred_one_hot_category = [ np.argmax(t) for t in y_pred_one_hot ]
                return confusion_matrix(y_dw_real_category, y_pred_one_hot_category)

        if monitor_method == 'mv':
            y_pred_labels, y_dw_real = MajorityVote(y_truth=self.y_real,
                                                    y_prediction=y_prediction,
                                                    s=self.segments_a_decision_window,
                                                    r=self.decision_overlap,
                                                    weights=value_coef)

            y_pred_one_hot = np.zeros_like(y_pred_labels)
            y_pred_one_hot[np.arange(len(y_pred_one_hot)), y_pred_labels.argmax(1)] = 1
            if monitor_measure == 'mse':
                loss_fn = tf.keras.losses.MeanSquaredError()
                return loss_fn(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'pesl':
                return pesl(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'pll':
                return pll(y_dw_real, y_pred_labels).numpy()
            elif monitor_measure == 'accuracy':
                return accuracy_score(y_dw_real, y_pred_one_hot)
            elif monitor_measure == 'precision':
                return precision_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'recall':
                return recall_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'f1':
                return f1_score(y_dw_real, y_pred_one_hot, average='macro')
            elif monitor_measure == 'confusionmatrix':
                y_dw_real_category = [ np.argmax(t) for t in y_dw_real ]
                y_pred_one_hot_category = [ np.argmax(t) for t in y_pred_one_hot ]
                return confusion_matrix(y_dw_real, y_pred_one_hot)

        return 0

In [86]:
class RestoringBest(tf.keras.callbacks.Callback):
    """
    :param monitor: '#METHOD_#MEASURE', #METHOD={'mv','ms'}, #MEASURE={'loss','accuracy','precision','recall','f1'}
    """

    def __init__(self, metric: ModelAnalyser, mode, monitor):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.metric = metric
        self.monitor = monitor
        self.mode = mode
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf
        self.best_weights = None
        self.best_epoch = -1

    def on_epoch_end(self, epoch, logs={}):
        current = self.metric.measurement(model=self.model, monitor=self.monitor)

        accuracy = self.metric.measurement(model=self.model, monitor="ms_accuracy")
        f1 = self.metric.measurement(model=self.model, monitor="ms_f1")

        print('epoch %d: \t %s: %f \t %s: %f \t %s: %f' % (epoch+1, self.monitor, current, "accuracy", accuracy, "f1", f1))

        monitor_measure = self.monitor.split('_')[1]

        if (self.mode == 'max' and current > self.best) or (self.mode == 'min' and current < self.best):
            self.best = current
            self.best_weights = self.model.get_weights()
            self.best_epoch = epoch

    def on_train_end(self, logs=None):
        if self.best_epoch > -1:
            print("Restoring model weights from the end of the %d epoch." % (self.best_epoch + 1))
            self.model.set_weights(self.best_weights)

# Evaluation

In [87]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

def analysis_model(loss_fn, loss_name, y_pred, y_real_raw, segment_size, segment_overlap, decision_size, decision_overlap):
    result = {'Core': {}, 'MV': {}, 'MS': {}}
    result['Core']['pesl'] = pesl(np.asarray(pd.get_dummies(y_real_raw), dtype=np.int8),
                                  np.asarray(y_pred, dtype=np.float64)).numpy()
    result['Core']['mse'] = tf.keras.metrics.mean_squared_error(np.asarray(pd.get_dummies(y_real_raw), dtype=np.int8),
                                  np.asarray(y_pred, dtype=np.float64)).numpy().mean()
    y_pred_arg = np.argmax(y_pred, axis=1)
    result['Core']['accuracy'] = accuracy_score(y_real_raw, y_pred_arg)
    result['Core']['precision'] = precision_score(y_real_raw, y_pred_arg, average='macro')
    result['Core']['recall'] = recall_score(y_real_raw, y_pred_arg, average='macro')
    result['Core']['f1'] = f1_score(y_real_raw, y_pred_arg, average='macro')

    segments_a_decision_window = get_segments_a_decision_window(segment_size, segment_overlap, decision_size)

    # Maximum Score
    y_pred_labels, y_real = AveragingProbabilities(y_truth=y_real_raw, y_prediction=y_pred,
                                                   s=segments_a_decision_window,
                                                   r=decision_overlap)

    result['MS']['pesl'] = pesl(y_real, y_pred_labels).numpy()
    result['MS']['mse'] = tf.keras.metrics.mean_squared_error(y_real, y_pred_labels).numpy().mean()
    temp = y_pred_labels.copy()
    y_pred_labels = np.zeros_like(temp)
    y_pred_labels[np.arange(len(temp)), temp.argmax(1)] = 1
    result['MS']['accuracy'] = accuracy_score(y_real, y_pred_labels)
    result['MS']['precision'] = precision_score(y_real, y_pred_labels, average='macro')
    result['MS']['recall'] = recall_score(y_real, y_pred_labels, average='macro')
    result['MS']['f1'] = f1_score(y_real, y_pred_labels, average='macro')

    # Majority Voting
    y_pred_labels, y_real = MajorityVote(y_truth=y_real_raw, y_prediction=y_pred,
                                         s=segments_a_decision_window,
                                         r=decision_overlap)

    result['MV']['pesl'] = pesl(y_real, y_pred_labels).numpy()
    result['MV']['mse'] = tf.keras.metrics.mean_squared_error(y_real, y_pred_labels).numpy().mean()
    temp = y_pred_labels.copy()
    y_pred_labels = np.zeros_like(temp)
    y_pred_labels[np.arange(len(temp)), temp.argmax(1)] = 1
    result['MV']['accuracy'] = accuracy_score(y_real, y_pred_labels)
    result['MV']['precision'] = precision_score(y_real, y_pred_labels, average='macro')
    result['MV']['recall'] = recall_score(y_real, y_pred_labels, average='macro')
    result['MV']['f1'] = f1_score(y_real, y_pred_labels, average='macro')

    return result

In [88]:
def get_segments_a_decision_window(segment_size, segment_overlap, decision_size):
    segment_overlap_size = segment_size * segment_overlap
    return int((decision_size - segment_size) / (segment_size - segment_overlap_size) + 1)

In [89]:
def AveragingProbabilities(y_truth, y_prediction, s, r):
    df = []
    y_dw_truth = []
    c = y_prediction.shape[1]
    r = int(np.floor(s * r))
    for _id in np.unique(y_truth):
        subset = y_prediction[np.where(y_truth == _id)]
        n = subset.shape[0]
        o = int(np.floor((n - r) / (s - r)))
        for i in range(o):
            row = np.zeros(c)
            for j in range(s):
                row += subset[(i * s) + j]
            df.append(row / s)
            y_dw_truth.append(_id)
    df = np.array(df)
    y_dw_truth = np.asarray(pd.get_dummies(y_dw_truth), dtype=np.int8)
    return df, y_dw_truth

In [90]:
def MajorityVote(y_truth, y_prediction, s, r):
    df = []
    y_dw_truth = []
    c = y_prediction.shape[1]
    r = int(np.floor(s * r))
    # Make prior prediction to one-hot
    y_categorical_pred = np.zeros_like(y_prediction)
    y_categorical_pred[np.arange(len(y_prediction)), y_prediction.argmax(1)] = 1

    for _id in np.unique(y_truth):
        subset = y_categorical_pred[np.where(y_truth == _id)]
        n = subset.shape[0]
        o = int(np.floor((n - r) / (s - r)))
        for i in range(o):
            row = np.zeros(c)
            for j in range(s):
                row += subset[(i * s) + j]
            df.append(row / s)
            y_dw_truth.append(_id)
    df = np.array(df)
    y_dw_truth = np.asarray(pd.get_dummies(y_dw_truth), dtype=np.int8)
    return df, y_dw_truth

# Train

In [91]:
def train_model(dataset: Dataset, classifier: Classifier, epochs, batch_size,
                log_dir, monitor_metric, monitor_mode, restore_best=True):

    callbacks = []
    tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    callbacks.append(tbCallBack)

    if restore_best:
        metric = ModelAnalyser(segment_size = classifier.segments_size,
                               segment_overlap = classifier.segments_overlap,
                               decision_size = classifier.decision_size,
                               decision_overlap = classifier.decision_overlap,
                               X = dataset.X_valid,
                               y = dataset.y_valid)
        restoring_best = RestoringBest(metric=metric, monitor=monitor_metric, mode=monitor_mode)
        callbacks.append(restoring_best)

    history, y_test_prediction = classifier.train(epochs=epochs,
                                                  X_train=dataset.X_train,
                                                  y_train=dataset.y_train,
                                                  X_valid=dataset.X_valid,
                                                  y_valid=dataset.y_valid,
                                                  X_test=dataset.X_test,
                                                  batch_size=batch_size,
                                                  callback=callbacks)

    result_test = analysis_model(loss_fn=classifier.get_loss_function(),
                                 loss_name=classifier.get_loss_function_name(),
                                 y_pred=y_test_prediction,
                                 y_real_raw=dataset.y_test,
                                 segment_size=classifier.segments_size,
                                 segment_overlap=classifier.segments_overlap,
                                 decision_size=classifier.decision_size,
                                 decision_overlap=classifier.decision_overlap)

    print('Classifier:%s Test(Core):%5.2f Test(MV):%5.2f ' % (str(classifier), result_test['Core']['accuracy'], result_test['MV']['accuracy']))

    return result_test

# Spliting

In [92]:
def h_block_analyzer(db_path, sample_rate, features, n_classes, noise_rate,
                     segments_time, segments_overlap,
                     decision_time, decision_overlap,
                     classifier, epochs, batch_size, data_length_time,
                     monitor_metric, monitor_mode, restore_best,
                     n_h_block, n_train_h_block, n_valid_h_block, n_test_h_block, h_moving_step=1):
    """
    :param db_path: the address of dataset directory
    :param sample_rate: the sampling rate of signals
    :param features: the signals of original data
    :param n_classes: the number of classes
    :param noise_rate: the rate of noises injected to test data
    :param segments_time: the length of each segment in seconds.
    :param segments_overlap: the overlap of each segment
    :param classifier: the neural network
    :param epochs: the number of training epochs
    :param batch_size: the number of segments in each batch
    :param data_length_time: the length of data of each class. -1 = whole.
    :param n_h_block: the number of all hv blocks
    :param n_train_h_block: the number of hv blocks to train network
    :param n_valid_h_block: the number of hv blocks to validate network
    :param n_test_h_block: the number of hv blocks to test network
    :param h_moving_step: the number of movement of test and validation blocks in each iteration
    :return:
    """

    final_statistics = {}
    add_noise = noise_rate < 100

    # Create hv blocks
    data_blocks = [i for i in range(n_h_block)]
    n_vt = (n_valid_h_block + n_test_h_block)
    n_iteration = int((n_h_block - n_vt) / h_moving_step)

    date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
    statistics = {}
    for i in range(n_iteration + 1):
        print('iteration: %d/%d' % (i + 1, n_iteration + 1))
        training_container = data_blocks[0:i] + data_blocks[i + n_vt:n_h_block]
        train_blocks = training_container[:n_train_h_block]
        valid_blocks = data_blocks[i: i + n_valid_h_block]
        test_blocks = data_blocks[i + n_valid_h_block: i + n_vt]
        dataset = Dataset(db_path,
                          sample_rate,
                          features=features,
                          window_time=segments_time,
                          window_overlap_percentage=segments_overlap,
                          decision_time=decision_time,
                          decision_overlap_percentage=decision_overlap,
                          add_noise=add_noise,
                          noise_rate=noise_rate,
                          train_blocks=train_blocks,
                          valid_blocks=valid_blocks,
                          test_blocks=test_blocks,
                          data_length_time=data_length_time)
        dataset.load_data(n_classes=n_classes, method=classifier.get_data_arrangement())
        logdir = os.path.join("./log/checkpointss/", date_str + "[" + str(i) + "]")
        if not os.path.exists(logdir):
            os.makedirs(logdir)
        result = train_model(dataset=dataset, classifier=classifier, epochs=epochs,
                             batch_size=batch_size,
                             restore_best=restore_best,
                             monitor_metric=monitor_metric,
                             monitor_mode=monitor_mode,
                             log_dir=logdir)

        for key in result.keys():
            if not key in statistics:
                statistics[key] = {}
            for inner_key in result[key].keys():
                if not inner_key in statistics[key]:
                    statistics[key][inner_key] = []
                statistics[key][inner_key].append(result[key][inner_key])
        shutil.rmtree(logdir, ignore_errors=True)

    print(statistics)
    return statistics

# Save Result

In [93]:
def save_result(log_dir, data: dict):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Save to file
    with open(log_dir + 'statistics.txt', 'a') as f:
        f.write('\n==========***==========\n')
        f.write(str(data))
        f.write('\n')

    csv_file = log_dir + 'statistics.csv'
    file_exists = os.path.isfile(csv_file)
    try:
        with open(csv_file, 'a') as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(data.keys())
            writer.writerow(data.values())
            csvfile.close()
    except IOError:
        print("I/O error")

In [94]:
def cumulative_average(numbers):
    avg = []
    for i in range(len(numbers)):
        check = 0
        l = i + 1
        for j in range(i+1):
            check += numbers[j]
        avg.append(check/l)
    return avg

# Datasets

In [95]:
problems = {'ConfLongDemo_JSI':{},
            'Healthy_Older_People':{},
            'User_Identification_From_Walking':{}
           }

problems['ConfLongDemo_JSI']['dataset'] = './datasets/ConfLongDemo_JSI/'
problems['ConfLongDemo_JSI']['n_classes'] = 5
problems['ConfLongDemo_JSI']['features'] = ["x", "y", "z"]
problems['ConfLongDemo_JSI']['sample_rate'] = 30
problems['ConfLongDemo_JSI']['data_length_time'] = -1
problems['ConfLongDemo_JSI']['n_h_block'] = 10
problems['ConfLongDemo_JSI']['n_train_h_block'] = 5
problems['ConfLongDemo_JSI']['n_valid_h_block'] = 2
problems['ConfLongDemo_JSI']['n_test_h_block'] = 3
problems['ConfLongDemo_JSI']['h_moving_step'] = 1
problems['ConfLongDemo_JSI']['decision_times'] = [3,4,5,6,7,8,9,10,30,60,2*60]
problems['ConfLongDemo_JSI']['segments_times'] = [3,4,5,6,7,8,9,10,30,60]

problems['Healthy_Older_People']['dataset'] = './datasets/Healthy_Older_People/'
problems['Healthy_Older_People']['n_classes'] = 12
problems['Healthy_Older_People']['features'] = ["X", "Y", "Z"]
problems['Healthy_Older_People']['sample_rate'] = 1
problems['Healthy_Older_People']['data_length_time'] = -1
problems['Healthy_Older_People']['n_h_block'] = 10
problems['Healthy_Older_People']['n_train_h_block'] = 5
problems['Healthy_Older_People']['n_valid_h_block'] = 2
problems['Healthy_Older_People']['n_test_h_block'] = 3
problems['Healthy_Older_People']['h_moving_step'] = 1
problems['Healthy_Older_People']['decision_times'] = [3,4,5,6,7,8,9,10,30,60,2*60,3*60]
problems['Healthy_Older_People']['segments_times'] = [3,4,5,6,7,8,9,10,30,60,2*60]

problems['User_Identification_From_Walking']['dataset'] = './datasets/User_Identification_From_Walking/'
problems['User_Identification_From_Walking']['n_classes'] = 22
problems['User_Identification_From_Walking']['features'] = [' x acceleration', ' y acceleration', ' z acceleration']
problems['User_Identification_From_Walking']['sample_rate'] = 32
problems['User_Identification_From_Walking']['data_length_time'] = -1
problems['User_Identification_From_Walking']['n_h_block'] = 10
problems['User_Identification_From_Walking']['n_train_h_block'] = 5
problems['User_Identification_From_Walking']['n_valid_h_block'] = 2
problems['User_Identification_From_Walking']['n_test_h_block'] = 3
problems['User_Identification_From_Walking']['h_moving_step'] = 1
problems['User_Identification_From_Walking']['decision_times'] = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
problems['User_Identification_From_Walking']['segments_times'] = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]


# Main Configuration

In [96]:
train_config = [
    {
        "loss_function": "categorical_crossentropy",
        "loss_metric": pesl,
        "monitor_metric": "ms_pesl",
        "monitor_mode": "min"
    }
]

# Configuration

In [100]:
noise_rate = 100
epochs = 20
batch_size = 32
models = ['CNN_L']
repetitions = 10
restore_best = True
tci=0

for problem in ['User_Identification_From_Walking','Healthy_Older_People', 'ConfLongDemo_JSI']:
    log_dir = f"./benchmark/logs/baselines/fusion/{problem}/"
    dataset = problems[problem]['dataset']
    n_classes = problems[problem]['n_classes']
    features = problems[problem]['features']
    sample_rate = problems[problem]['sample_rate']
    data_length_time = problems[problem]['data_length_time']
    n_h_block = problems[problem]['n_h_block']
    n_train_h_block = problems[problem]['n_train_h_block']
    n_valid_h_block = problems[problem]['n_valid_h_block']
    n_test_h_block = problems[problem]['n_test_h_block']
    h_moving_step = problems[problem]['h_moving_step']
    decision_times = problems[problem]['decision_times']
    segments_times = problems[problem]['segments_times']
    
    for model in models:
        for decision_time in decision_times:
            for decision_overlap in [0.0]:
                for segments_time in segments_times:
                    for segments_overlap in [0.75]:
                        if float(decision_time*0.75) < float(segments_time):
                            continue
                        
                        if (os.path.exists(log_dir + "statistics.csv")):
                            is_investigated = False
                            saved_statistics = pd.read_csv(log_dir + "statistics.csv")
                            for index, row in saved_statistics.iterrows():
                                if row['dataset'] == str(dataset) and row['inner_classifier'] == str(model):
                                    t1 = datetime.strptime(row['segments_time'],"%H:%M:%S")
                                    t2 = datetime.strptime(row['decision_time'],"%H:%M:%S")
                                    td1 = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
                                    td2 = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
                                    if (td1.total_seconds() == int(segments_time) 
                                        and td2.total_seconds() == int(decision_time)):
                                        print("dataset: " + problem + " " +
                                              "model: " + model + " " +
                                            "dl: " + row['decision_time'] + " and wl:" + row[
                                            'segments_time'] + " is investigated!")
                                        is_investigated = True
                                        break
                            if (is_investigated):
                                continue

                        print("dl: " + str(timedelta(seconds=int(decision_time))) +
                              " and " +
                              "wl: " + str(timedelta(seconds=int(segments_time))))
                        
                        classifier = eval(model)(classes=n_classes,
                                                 n_features=len(features),
                                                 segments_size=int(segments_time * sample_rate),
                                                 segments_overlap=segments_overlap,
                                                 decision_size=int(decision_time * sample_rate),
                                                 decision_overlap=decision_overlap,
                                                 loss_metric=train_config[0]["loss_metric"],
                                                 loss_function=train_config[0]["loss_function"])
                        # cross-validation
                        start = datetime.now()
                        statistics = h_block_analyzer(db_path=dataset,
                                                      sample_rate=sample_rate,
                                                      features=features,
                                                      n_classes=n_classes,
                                                      noise_rate=noise_rate,
                                                      segments_time=segments_time,
                                                      segments_overlap=segments_overlap,
                                                      decision_time=decision_time,
                                                      decision_overlap=decision_overlap,
                                                      classifier=classifier,
                                                      epochs=epochs,
                                                      batch_size=batch_size,
                                                      data_length_time=data_length_time,
                                                      n_h_block=n_h_block,
                                                      n_train_h_block=n_train_h_block,
                                                      n_valid_h_block=n_valid_h_block,
                                                      n_test_h_block=n_test_h_block,
                                                      h_moving_step=h_moving_step,
                                                      monitor_metric=train_config[0]["monitor_metric"],
                                                      monitor_mode=train_config[0]["monitor_mode"],
                                                      restore_best=restore_best)
                        end = datetime.now()
                        running_time = end - start
                        # Summarizing the results of cross-validation
                        data = {}
                        data['dataset'] = dataset
                        data['class'] = str(n_classes)
                        loss_metric = train_config[0]["loss_metric"]
                        data['loss_metric'] = loss_metric if type(loss_metric) == str else loss_metric.__name__
                        loss_function = train_config[0]["loss_function"]
                        data['loss_function'] = loss_function if type(loss_function) == str else loss_function.__name__
                        data['monitor_metric'] = train_config[0]["monitor_metric"]
                        data['monitor_mode'] = train_config[0]["monitor_mode"]
                        data['restore_best'] = str(restore_best)
                        data['features'] = str(features)
                        data['sample_rate'] = str(sample_rate)
                        data['noise_rate'] = str(noise_rate)
                        data['epochs'] = str(epochs)
                        data['batch_size'] = str(batch_size)
                        data['data_length_time'] = str(data_length_time)
                        data['repetitions'] = str(repetitions)
                        data['n_h_block'] = str(n_h_block)
                        data['n_train_h_block'] = str(n_train_h_block)
                        data['n_valid_h_block'] = str(n_valid_h_block)
                        data['n_test_h_block'] = str(n_test_h_block)
                        data['h_moving_step'] = str(h_moving_step)
                        data['segments_time'] = str(segments_time)
                        data['segments_overlap'] = str(segments_overlap)
                        data['inner_classifier'] = str(model)
                        data['datetime'] = datetime.now().strftime("%Y:%m:%d %H:%M:%S")
                        data['running_time'] = str(running_time.seconds) + " seconds"
                        data['n_params'] = classifier.count_params()
                        data['segments_time'] = timedelta(seconds=int(segments_time))
                        data['segments_overlap'] = segments_overlap
                        data['decision_time'] = timedelta(seconds=int(decision_time))
                        data['decision_overlap'] = decision_overlap
                        statistics_summary = {}
                        for key in statistics.keys():
                            for inner_key in statistics[key].keys():
                                statistics_summary[key + '_' + inner_key + '_mean'] = np.average(statistics[key][inner_key])
                                statistics_summary[key + '_' + inner_key + '_std'] = np.std(statistics[key][inner_key])
                                statistics_summary[key + '_' + inner_key + '_max'] = np.max(statistics[key][inner_key])
                                statistics_summary[key + '_' + inner_key + '_min'] = np.min(statistics[key][inner_key])
                        data.update(statistics_summary)
                        # Save information
                        save_result(log_dir=log_dir, data=data)

dl: 0:00:04 and wl: 0:00:03
iteration: 1/6
./User_Identification_From_Walking/segments/method: 3d/noise: 100/n_classes:22/wl: 96/wo: 72/dl: 128/do: 0/train: [5, 6, 7, 8, 9]/valid: [0, 1]/test: [2, 3, 4]/
Dataset is already
Epoch 1/20


2024-01-14 04:14:29.421762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 